<a href="https://colab.research.google.com/github/Yash-1812/Advance_Python/blob/main/DRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###
!pip install -q transformers sentencepiece scikit-learn
!pip install -q torch --upgrade

In [ ]:
!pip install -q pdfplumber

In [ ]:
from google.colab import files
from pathlib import Path
import zipfile
import shutil


DATA_DIR = Path("lectures")
DATA_DIR.mkdir(exist_ok=True)


print("Upload a ZIP folder containing all course files:")
uploaded = files.upload()


zip_path = list(uploaded.keys())[0]


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR)

print(f"Extracted all files from '{zip_path}' into '{DATA_DIR}/'")


all_files = list(DATA_DIR.rglob("*"))
print(f"Files found in {DATA_DIR}/: {all_files}")

Upload a ZIP folder containing all course files:


Saving DRM.zip to DRM (1).zip
Extracted all files from 'DRM (1).zip' into 'lectures/'
Files found in lectures/: [PosixPath('lectures/DRM'), PosixPath('lectures/DRM/3_ Intro to Derivatives.pdf'), PosixPath('lectures/DRM/2_ Risk and Return.pdf'), PosixPath('lectures/DRM/5_ Futures Pricing.pdf'), PosixPath('lectures/DRM/8_ Options Trading Strategy.pdf'), PosixPath('lectures/DRM/4_ Futures.pdf'), PosixPath('lectures/DRM/10_ Black Scholes Model.pdf'), PosixPath('lectures/DRM/9_ Binomial Model.pdf'), PosixPath('lectures/DRM/6_ Hedging using Futures.pdf'), PosixPath('lectures/DRM/1_ TVM.pdf'), PosixPath('lectures/DRM/7_ Options.pdf')]


In [ ]:
###
import pdfplumber
from pathlib import Path
import json

DATA_DIR = Path("lectures")  # Folder containing slides + textbook
all_texts = []

def extract_from_pdf(pdf_path):
    """Extract text from non-scanned PDFs."""
    pages = []
    with pdfplumber.open(str(pdf_path)) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ''
            if text.strip():
                pages.append({
                    'file': pdf_path.name,
                    'page_num': i,
                    'text': text
                })
    return pages

# Collect all PDFs
all_files = list(DATA_DIR.rglob("*.pdf"))
print(f"Found {len(all_files)} PDF files.")

for f in all_files:
    pdf_text = extract_from_pdf(f)
    all_texts.extend(pdf_text)

# Save all text
with open("all_course_material.json", "w", encoding="utf-8") as f:
    json.dump(all_texts, f, ensure_ascii=False, indent=2)

print(f"Extracted {len(all_texts)} pages/slides.")

Found 10 PDF files.


Extracted 347 pages/slides.


In [ ]:
from google.colab import files
import zipfile

print(" Upload your ZIP file containing scanned paper images (JPG, PNG, etc).")
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith(".zip"):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("data/previous_papers")
        print(f" Extracted {filename} → data/previous_papers/")

 Upload your ZIP file containing scanned paper images (JPG, PNG, etc).


Saving DRM_Questions.zip to DRM_Questions.zip
 Extracted DRM_Questions.zip → data/previous_papers/


In [ ]:
###
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine page text into a list
documents = [page['text'] for page in all_texts]

# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(documents)

print("TF-IDF matrix created.")

TF-IDF matrix created.


In [ ]:
###
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

MODEL_NAME = "tiiuae/falcon-7b-instruct"  # Free, instruction-tuned

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype=torch.float16)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,  # adjust for longer/shorter answers
    temperature=0.7
)

print("LLM loaded and ready.")

In [ ]:
###
import numpy as np

def retrieve_context(query, top_k=5):
    """Retrieve top-k relevant pages using TF-IDF cosine similarity."""
    q_vec = tfidf.transform([query])
    scores = (tfidf_matrix @ q_vec.T).toarray().squeeze()
    top_indices = np.argsort(scores)[-top_k:][::-1]
    contexts = [documents[i] for i in top_indices]
    return "\n\n".join(contexts)

def answer_question(query):
    """Retrieve context and generate answer using LLM."""
    context = retrieve_context(query)
    prompt = f"Use the following context to answer the question:\n{context}\n\nQuestion: {query}\nAnswer:"
    output = llm_pipeline(prompt, max_new_tokens=200, do_sample=True)[0]['generated_text']
    # Remove prompt from output
    answer = output.split("Answer:")[-1].strip()
    return answer
while True:
  query = input("Enter your query or enter exit")
  if query == "exit":
    break
  print(answer_question(query))

Enter your query or enter exitExplain forward contract


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


A forward contract is a
financial contract between two parties
involving the sale and purchase of the
same asset (goods or services) at an
agreed-upon price, at a future date,
known as the "contract date".
The parties involved in a forward
contract are called "counterparties" and
they enter into the contract with the
understanding that the contract will be
fulfilled. Forward contracts are
instruments used to hedge against price
risks in the future.

The price of the contract is derived by
multiplying the interest rate by the
difference in the forward prices of the
goods being sold and those being
bought. Forward contracts are OTC
(over-the-counter) contracts that are
traded on a bilateral basis between two
parties.
Enter your query or enter exitWhat are the 4 options positions


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Long call, Long put, Short call, Short put
Options Markets
1

Hedging
One of the most important and practical

applications of Futures and Options is
‘Hedging’. In the event of any adverse
market movements, hedging is a simple
work around to protect your trading
positions from making a loss.
19

Extensions of Put-Call Parity
American options; D = 0

S − K < C − P < S − Ke−rT
0 0
European options; D > 0

c + D + Ke −rT = p + S
0
American options; D > 0

S − D − K < C − P < S − Ke −rT
0 0
42

Bounds for European and American Put
Options (No Dividends)
40

The following table shows
Enter your query or enter exitexit
